<a href="https://colab.research.google.com/github/timduignan/orb-models/blob/main/examples/OrbMDTut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First you need to enable the gpu under Runtime, change runtime type. Then pip install the relevant libraries:

**Note:** This tutorial uses the new OrbMol models which require charge and spin multiplicity specification.

In [ ]:
!pip install -q orb-models

Then we need to get the example input file and run script from the github repo. We'll use the updated version that includes the new OrbMol models:

In [ ]:
!wget https://raw.githubusercontent.com/orbital-materials/orb-models/main/examples/NaClWater.xyz
!wget https://raw.githubusercontent.com/orbital-materials/orb-models/main/examples/NaClWaterMD.py

--2025-01-16 08:01:38--  https://raw.githubusercontent.com/orbital-materials/orb-models/main/examples/*
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-01-16 08:01:38 ERROR 404: Not Found.



In [ ]:
!ls

md_nvt.log  NaClWaterMD.py  NaClWaterMD.py.1  NaClWaterMD.xyz  NaClWater.xyz  sample_data


Then simply run the python script.

In [ ]:
!python NaClWaterMD.py

Using device: cuda
/usr/local/lib/python3.11/dist-packages/orb_models/forcefield/pretrained.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(loca

The simulation runs using the new orb-v3-conservative-omol model with proper charge and spin multiplicity specification. It generates 100 frames of simulation and should be stable at 300 K:

In [ ]:
!head md_nvt.log

tail: cannot open 'md_nvt.log' for reading: No such file or directory


The trajectory is stored in NaClWaterMD.xyz. We can visualise the a frame of the simulation:

In [ ]:
pip install py3Dmol

In [ ]:
import py3Dmol

# Function to extract a specific frame from the .xyz file
def extract_frame(xyz_file, frame_number):
    with open(xyz_file, "r") as f:
        lines = f.readlines()

    # Parse the file
    num_atoms = int(lines[0].strip())  # Number of atoms from the first line
    frame_start = frame_number * (num_atoms + 2)  # Start of the desired frame
    frame_lines = lines[frame_start : frame_start + num_atoms + 2]  # Extract frame
    return "".join(frame_lines)


# Specify the file and the frame number
frame_number = 3  # Pick the frame (0-based indexing)
xyz_file = "NaClWaterMD.xyz"
frame_data = extract_frame(xyz_file, frame_number)

# Visualize the extracted frame with balls and sticks
viewer = py3Dmol.view(width=800, height=400)
viewer.addModel(frame_data, "xyz")  # Add the trajectory frame
viewer.setStyle({
    "stick": {},                      # Default stick style
    "sphere": {"radius": 0.5}        # Sphere style with adjustable radius
})
viewer.zoomTo()
viewer.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

You can run longer simulations at higher temperatures to test stability. The new OrbMol models should provide better accuracy for molecular systems. Remember to update the cell size parameter for different systems and adjust print frequencies as needed:

**Note:** If you want to use your own molecular systems, make sure to set appropriate charge and spin multiplicity values in your input handling code.

In [ ]:
from NaClWaterMD import run_md_simulation

# Run with custom parameters
run_md_simulation(
    input_file="NaClWater.xyz",
    cell_size= 25.25,
    temperature_K=400,
    total_steps=10000,
    traj_interval = 20,
    log_interval=20
)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/orb_models/forcefield/pretrained.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(local_path, map_locatio

You probably want to load a conda environment or similar with

```
conda create --name orb_env python=3.11
conda activate orb_env

```

Please reach out if you have any questions or issues. My email is tim@orbitalmaterials.com
Happy simulating!